In [2]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT, OMNICELL_ROOT
from omnicell.models.selector import load_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml(f"{OMNICELL_ROOT}/configs/models/scot/proportional_scot.yaml")
ETL_CONFIG = ETLConfig.from_yaml(f"{OMNICELL_ROOT}/configs/ETL/no_preproc_drop_unmatched_scot.yaml")
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml(f"{OMNICELL_ROOT}/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml(f"{OMNICELL_ROOT}/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)




#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.X.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")



2025-02-17 12:52:06,873 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/FrangiehIzar2021_protein.yaml
2025-02-17 12:52:06,875 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/PapalexiSatija2021_eccite_arrayed_RNA.yaml
2025-02-17 12:52:06,876 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/AdamsonWeissman2016_GSM2406677_10X005.yaml
2025-02-17 12:52:06,877 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/PapalexiSatija2021_eccite_protein.yaml
2025-02-17 12:52:06,879 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/SchraivogelSteinmetz2020_TAP_SCREEN__chromosome_11_screen.yaml
2025-02-17 12:52:06,880 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/DatlingerBock2021.yaml
2025-02-17 12:52:06,881 - INFO - Loading data

Using device: cuda


2025-02-17 12:52:11,209 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-02-17 12:52:11,210 - INFO - Preprocessing training data
2025-02-17 12:52:11,212 - INFO - Using identity features for perturbations
2025-02-17 12:52:11,327 - INFO - Removing observations with perturbations not in the dataset as a column
2025-02-17 12:52:11,521 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
2025-02-17 12:52:41,004 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [5]:

adata.uns['synthetic_data_paths']

[PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_14.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_13.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_8.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_1.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_6.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/syn

In [ ]:
model = get_model(config.model_config.name, config.model_config.parameters, loader, pert_rep_map, input_dim, device, pert_ids)

In [3]:
model.max_epochs = 10

In [ ]:
model.train(adata, None)

Creating source indices
Creating target indices
Creating pert indices
Creating source and target dicts


In [7]:
import numpy as np

logger.info("Running evaluation")

# evaluate each pair of cells and perts
eval_dict = {}
for cell_id, pert_id, ctrl_data, gt_data in loader.get_eval_data():
    logger.debug(f"Making predictions for cell: {cell_id}, pert: {pert_id}")

    preds = model.make_predict(ctrl_data, pert_id, cell_id)
    eval_dict[(cell_id, pert_id)] = (ctrl_data.X.toarray(), gt_data.X.toarray(), preds)
    break
    
if not config.etl_config.log1p:
    for (cell, pert) in eval_dict:  
        ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]
        # normalize to sum to 1
        ctrl_data = ctrl_data / ctrl_data.sum(axis=1).reshape(-1, 1) * 10_000
        gt_data = gt_data / gt_data.sum(axis=1).reshape(-1, 1) * 10_000
        pred_pert = pred_pert / pred_pert.sum(axis=1).reshape(-1, 1) * 10_000
        eval_dict[(cell, pert)] =  (np.log1p(ctrl_data), np.log1p(gt_data), np.log1p(pred_pert))


2025-02-06 20:59:38,409 - INFO - Running evaluation
2025-02-06 20:59:38,411 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-02-06 20:59:38,413 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-02-06 20:59:38,414 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-02-06 20:59:38,416 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-02-06 20:59:38,417 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-02-06 20:59:38,418 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-02-06 20:59:38,420 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/

In [8]:
import scanpy as sc
from omnicell.evaluation.utils import get_DEGs, get_eval, get_DEG_Coverage_Recall, get_DEGs_overlaps
pval_threshold = 0.05
log_fold_change_threshold = 0.0

results_dict = {}
for (cell, pert) in eval_dict:  
    ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]

    pred_pert = sc.AnnData(X=pred_pert)
    true_pert = sc.AnnData(X=gt_data)
    control = sc.AnnData(X=ctrl_data)

    logger.debug(f"Getting ground Truth DEGs for {pert} and {cell}")
    true_DEGs_df = get_DEGs(control, true_pert)
    signif_true_DEG = true_DEGs_df[true_DEGs_df['pvals_adj'] < pval_threshold]

    logger.debug(f"Number of significant DEGS from ground truth: {signif_true_DEG.shape[0]}")

    logger.debug(f"Getting predicted DEGs for {pert} and {cell}")
    pred_DEGs_df = get_DEGs(control, pred_pert)


    logger.debug(f"Getting evaluation metrics for {pert} and {cell}")
    r2_and_mse = get_eval(control, true_pert, pred_pert, true_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    logger.debug(f"Getting DEG overlaps for {pert} and {cell}")
    DEGs_overlaps = get_DEGs_overlaps(true_DEGs_df, pred_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    results_dict[(cell, pert)] = (r2_and_mse, DEGs_overlaps)



/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
for (cell, pert) in results_dict:
    r2_and_mse, DEGs_overlaps = results_dict[(cell, pert)]
    print(f"Cell: {cell}, Pert: {pert}")
    # print(f"R2 and MSE: {r2_and_mse}")
    print(f"DEGs Overlaps: {DEGs_overlaps}")
    print("-"*100)

Cell: k562, Pert: RPL15
DEGs Overlaps: {'Overlap_in_top_2157_DEGs': 1384, 'Overlap_in_top_100_DEGs': 67, 'Overlap_in_top_50_DEGs': 26, 'Overlap_in_top_20_DEGs': 8, 'Jaccard': 0.3225588085970553}
----------------------------------------------------------------------------------------------------


In [11]:
r2_and_mse


{'all_genes_mean_sub_diff_R': np.float32(0.9394147),
 'all_genes_mean_sub_diff_R2': np.float32(0.88249993),
 'all_genes_mean_sub_diff_MSE': np.float32(0.0017594295),
 'all_genes_mean_fold_diff_R': np.float32(0.9819075),
 'all_genes_mean_fold_diff_R2': np.float32(0.9641423),
 'all_genes_mean_fold_diff_MSE': np.float32(0.10515996),
 'all_genes_mean_R': np.float32(0.99663806),
 'all_genes_mean_R2': np.float32(0.99328744),
 'all_genes_mean_MSE': np.float32(0.0017594295),
 'all_genes_var_R': np.float32(0.914543),
 'all_genes_var_R2': np.float32(0.8363888),
 'all_genes_var_MSE': np.float32(0.0018092245),
 'all_genes_corr_mtx_R': np.float64(0.17428146307689132),
 'all_genes_corr_mtx_R2': np.float64(0.030374028372221834),
 'all_genes_corr_mtx_MSE': np.float64(0.007376010796250857),
 'all_genes_cov_mtx_R': np.float64(0.23831284114404422),
 'all_genes_cov_mtx_R2': np.float64(0.05679301025414645),
 'all_genes_cov_mtx_MSE': np.float64(0.00021684922227372595),
 'Top_2157_DEGs_sub_diff_R': np.float3